In [52]:
import requests
from bs4 import BeautifulSoup
from datetime import date,datetime
import re

In [53]:
def pageDetail(detailurl):
    res = requests.get(detailurl)
    soup = BeautifulSoup(res.text, 'html.parser')
    i = 1
    push = ""
    
    if len(soup.select('#main-content')[0].contents) > 4:
        # 抓取內文
        summary = soup.select('#main-content')[0].contents[4]
        # 抓取時間
        time = soup.select('.article-meta-value')[3].text
        dt =datetime.strptime(time, '%a %b %d %H:%M:%S %Y')
        
        if (len(soup.select('#main-content')[0].select('.push')) > 0):
            for push_text in soup.select('#main-content')[0].select('.push'):
                push = push + str(i) + push_text.text
                i = i + 1
        # 以字典回傳資料
        return {'summary': summary, 'dt': dt, 'push': push}

#d = pageDetail('https://www.ptt.cc/bbs/creditcard/M.1469180146.A.1D0.html')

In [54]:
home_page = 'https://www.ptt.cc/bbs/Salary/index.html'
home_section = requests.get(home_page)
home_soup = BeautifulSoup(home_section.text, 'html.parser')

controls = home_soup.find('div', 'btn-group-paging').find_all('a', 'btn')

link = controls[1].get('href')
max_page = int(re.findall('\d+', link)[0]) + 1
min_page = 2375
#min_page = 1

In [55]:
domain = 'https://www.ptt.cc'
#建立空 List
pttary = []
for page in range(max_page,min_page,-1):
    #紀錄一下跑到哪
    print(page)
    
    ptturl = 'https://www.ptt.cc/bbs/Salary/index{}.html'.format(page)
    #使用format 更新頁數
    section = requests.get(ptturl.format(page))
    #將資料讀取至BeautifulSoup
    soup = BeautifulSoup(section.text, 'html.parser')
    #分別抓取各標籤元素
    for ent in soup.select('.r-ent'): 
        if len(ent.select('a')) >0:
            dic = {}
            dic['title'] = ent.select('.title')[0].text
            dic['link'] = domain + ent.select('a')[0]['href']
            #更新內容頁抓取到的資料
            dic.update(pageDetail(dic['link']))
            pttary.append(dic)

2380
2379
2378
2377
2376


In [48]:
import pandas
df = pandas.DataFrame(pttary)
df.head()

,dt,link,push,summary,title
0,2017-04-22 10:28:21,https://www.ptt.cc/bbs/Salary/M.1492828105.A.1...,1推 kynoko: 從政如果老闆扶搖直上真的有入閣機會，看你要不要賭 04/22 10:3...,A 公司(現職) 建設公司採購4年 年薪60-70(南部) 工作穩定 ...,\n[問題] 建設公司採購 VS 議員候選人助理\n
1,2017-04-22 12:07:24,https://www.ptt.cc/bbs/Salary/M.1492834047.A.3...,1推 stevenju: 我會去啦 去了跳槽機會也多 04/22 12:15\n2→ ste...,長駐三年 有租公寓一層...一人一間房間.. 公司包住包水電，上班往返有交通車 宿舍...,\n[問題] 公司在找人外派新加坡..\n
2,2017-04-22 12:34:59,https://www.ptt.cc/bbs/Salary/M.1492835703.A.F...,1推 timxdada: 台灣紡織夕陽產業，話至此剩下自己領悟吧 04/22 12:55\n...,想請問台南的台南紡織，也就是簡稱的南紡， 在儲備幹部這個職位的環境安全性如何，是與機械相...,\n[問題] 南紡工作安全問題\n
3,2017-04-22 12:35:46,https://www.ptt.cc/bbs/Salary/M.1492835748.A.1...,1→ HAPPY4ARIEL: tax板 04/22 12:48\n2→ HAPPY4AR...,不好意思因為不知道該去哪個版問，如果有正確的版還請大家分享一下 不會報稅，想請問腦袋清楚...,\n[問題] 想問繳稅問題\n
4,2017-04-22 13:30:39,https://www.ptt.cc/bbs/Salary/M.1492839042.A.8...,1推 JFLung9536: 因為你不是他的人阿 04/22 13:57\n2噓 yamak...,小弟認為有些嚴重，不曉得如何處理，以前未曾碰過類似情形故來請教網友。 現任直屬主管是協...,\n[問題] 越級指揮的主管如何共事？\n


In [58]:
#print(pttary[5]['push'])
len(pttary)
with open('df.json', 'w', encoding='utf-8') as file:
    df.to_json(file, force_ascii=False)
df.to_excel('df.xlsx')